In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as nm
import requests
import json

In [91]:
api_key = "ZOC333QF0ZHXU5YD"
series = "TIME_SERIES_DAILY"
request_url = "https://www.alphavantage.co/query"

In [92]:
def fetch_stock_values(symbol):
    query_params = {
        'function' : series,
        'symbol' : symbol,
        'apikey': api_key
    }
    headers = {
        'User-Agent': 'XY',
        'Content-type': 'application/json'
    }
    response = requests.get(request_url, headers=headers, params=query_params)
    return response.text if response.status_code == 200 else "Something went wrong"

In [93]:
# SPDR S&P 500 ETF Trust
spy_json = json.loads(fetch_stock_values(symbol='spy'))
spy_df = pd.DataFrame(spy_json['Time Series (Daily)'])
spy_df

,2022-04-22,2022-04-21,2022-04-20,2022-04-19,2022-04-18,2022-04-14,2022-04-13,2022-04-12,2022-04-11,2022-04-08,...,2021-12-13,2021-12-10,2021-12-09,2021-12-08,2021-12-07,2021-12-06,2021-12-03,2021-12-02,2021-12-01,2021-11-30
1. open,436.9100,448.5400,446.9200,437.8600,436.8100,443.5500,438.0300,443.0800,444.1100,447.9700,...,470.1900,469.2300,468.1500,468.7000,464.4100,456.1300,459.1700,450.7300,461.6400,462.0000
2. high,438.0825,450.0100,447.5700,445.8000,439.7500,444.7301,444.1100,445.7500,445.0000,450.6300,...,470.5600,470.9000,469.6291,470.0000,468.8800,460.7900,460.3000,459.0700,464.6700,464.0300
3. low,425.4400,437.1000,443.4800,437.6800,435.6100,437.6800,437.8400,436.6501,439.3900,445.9400,...,466.2700,466.5100,466.1400,466.8300,458.6546,453.5600,448.9200,450.3100,450.2900,455.3000
4. close,426.0400,438.0600,444.7100,445.0400,437.9700,437.7900,443.3100,438.2900,439.9200,447.5700,...,466.5700,470.7400,466.3500,469.5200,468.2800,458.7900,453.4200,457.4000,450.5000,455.5600
5. volume,132471772,85417327,64140001,77821013,66002498,97869451,74070394,84363635,89770538,79272711,...,87724680,77159757,61272568,72238777,95484688,98977532,137331647,127637758,132485835,148559598


In [94]:
# Zoom Video Communications Inc
zm_json = json.loads(fetch_stock_values(symbol='ZM'))
zm_df = pd.DataFrame(spy_json['Time Series (Daily)'])
zm_df

,2022-04-22,2022-04-21,2022-04-20,2022-04-19,2022-04-18,2022-04-14,2022-04-13,2022-04-12,2022-04-11,2022-04-08,...,2021-12-13,2021-12-10,2021-12-09,2021-12-08,2021-12-07,2021-12-06,2021-12-03,2021-12-02,2021-12-01,2021-11-30
1. open,436.9100,448.5400,446.9200,437.8600,436.8100,443.5500,438.0300,443.0800,444.1100,447.9700,...,470.1900,469.2300,468.1500,468.7000,464.4100,456.1300,459.1700,450.7300,461.6400,462.0000
2. high,438.0825,450.0100,447.5700,445.8000,439.7500,444.7301,444.1100,445.7500,445.0000,450.6300,...,470.5600,470.9000,469.6291,470.0000,468.8800,460.7900,460.3000,459.0700,464.6700,464.0300
3. low,425.4400,437.1000,443.4800,437.6800,435.6100,437.6800,437.8400,436.6501,439.3900,445.9400,...,466.2700,466.5100,466.1400,466.8300,458.6546,453.5600,448.9200,450.3100,450.2900,455.3000
4. close,426.0400,438.0600,444.7100,445.0400,437.9700,437.7900,443.3100,438.2900,439.9200,447.5700,...,466.5700,470.7400,466.3500,469.5200,468.2800,458.7900,453.4200,457.4000,450.5000,455.5600
5. volume,132471772,85417327,64140001,77821013,66002498,97869451,74070394,84363635,89770538,79272711,...,87724680,77159757,61272568,72238777,95484688,98977532,137331647,127637758,132485835,148559598


In [95]:
# Delta Air Lines, Inc
dal_json = json.loads(fetch_stock_values(symbol='DAL'))
dal_df = pd.DataFrame(spy_json['Time Series (Daily)'])
dal_df

,2022-04-22,2022-04-21,2022-04-20,2022-04-19,2022-04-18,2022-04-14,2022-04-13,2022-04-12,2022-04-11,2022-04-08,...,2021-12-13,2021-12-10,2021-12-09,2021-12-08,2021-12-07,2021-12-06,2021-12-03,2021-12-02,2021-12-01,2021-11-30
1. open,436.9100,448.5400,446.9200,437.8600,436.8100,443.5500,438.0300,443.0800,444.1100,447.9700,...,470.1900,469.2300,468.1500,468.7000,464.4100,456.1300,459.1700,450.7300,461.6400,462.0000
2. high,438.0825,450.0100,447.5700,445.8000,439.7500,444.7301,444.1100,445.7500,445.0000,450.6300,...,470.5600,470.9000,469.6291,470.0000,468.8800,460.7900,460.3000,459.0700,464.6700,464.0300
3. low,425.4400,437.1000,443.4800,437.6800,435.6100,437.6800,437.8400,436.6501,439.3900,445.9400,...,466.2700,466.5100,466.1400,466.8300,458.6546,453.5600,448.9200,450.3100,450.2900,455.3000
4. close,426.0400,438.0600,444.7100,445.0400,437.9700,437.7900,443.3100,438.2900,439.9200,447.5700,...,466.5700,470.7400,466.3500,469.5200,468.2800,458.7900,453.4200,457.4000,450.5000,455.5600
5. volume,132471772,85417327,64140001,77821013,66002498,97869451,74070394,84363635,89770538,79272711,...,87724680,77159757,61272568,72238777,95484688,98977532,137331647,127637758,132485835,148559598


In [96]:
# Annaly Capital Management, Inc
nly_json = json.loads(fetch_stock_values(symbol='NLY'))
nly_df = pd.DataFrame(spy_json['Time Series (Daily)'])
nly_df

,2022-04-22,2022-04-21,2022-04-20,2022-04-19,2022-04-18,2022-04-14,2022-04-13,2022-04-12,2022-04-11,2022-04-08,...,2021-12-13,2021-12-10,2021-12-09,2021-12-08,2021-12-07,2021-12-06,2021-12-03,2021-12-02,2021-12-01,2021-11-30
1. open,436.9100,448.5400,446.9200,437.8600,436.8100,443.5500,438.0300,443.0800,444.1100,447.9700,...,470.1900,469.2300,468.1500,468.7000,464.4100,456.1300,459.1700,450.7300,461.6400,462.0000
2. high,438.0825,450.0100,447.5700,445.8000,439.7500,444.7301,444.1100,445.7500,445.0000,450.6300,...,470.5600,470.9000,469.6291,470.0000,468.8800,460.7900,460.3000,459.0700,464.6700,464.0300
3. low,425.4400,437.1000,443.4800,437.6800,435.6100,437.6800,437.8400,436.6501,439.3900,445.9400,...,466.2700,466.5100,466.1400,466.8300,458.6546,453.5600,448.9200,450.3100,450.2900,455.3000
4. close,426.0400,438.0600,444.7100,445.0400,437.9700,437.7900,443.3100,438.2900,439.9200,447.5700,...,466.5700,470.7400,466.3500,469.5200,468.2800,458.7900,453.4200,457.4000,450.5000,455.5600
5. volume,132471772,85417327,64140001,77821013,66002498,97869451,74070394,84363635,89770538,79272711,...,87724680,77159757,61272568,72238777,95484688,98977532,137331647,127637758,132485835,148559598


In [97]:
# GOLD
gld_json = json.loads(fetch_stock_values(symbol='GOLD'))
gld_df = pd.DataFrame(spy_json['Time Series (Daily)'])
gld_df

,2022-04-22,2022-04-21,2022-04-20,2022-04-19,2022-04-18,2022-04-14,2022-04-13,2022-04-12,2022-04-11,2022-04-08,...,2021-12-13,2021-12-10,2021-12-09,2021-12-08,2021-12-07,2021-12-06,2021-12-03,2021-12-02,2021-12-01,2021-11-30
1. open,436.9100,448.5400,446.9200,437.8600,436.8100,443.5500,438.0300,443.0800,444.1100,447.9700,...,470.1900,469.2300,468.1500,468.7000,464.4100,456.1300,459.1700,450.7300,461.6400,462.0000
2. high,438.0825,450.0100,447.5700,445.8000,439.7500,444.7301,444.1100,445.7500,445.0000,450.6300,...,470.5600,470.9000,469.6291,470.0000,468.8800,460.7900,460.3000,459.0700,464.6700,464.0300
3. low,425.4400,437.1000,443.4800,437.6800,435.6100,437.6800,437.8400,436.6501,439.3900,445.9400,...,466.2700,466.5100,466.1400,466.8300,458.6546,453.5600,448.9200,450.3100,450.2900,455.3000
4. close,426.0400,438.0600,444.7100,445.0400,437.9700,437.7900,443.3100,438.2900,439.9200,447.5700,...,466.5700,470.7400,466.3500,469.5200,468.2800,458.7900,453.4200,457.4000,450.5000,455.5600
5. volume,132471772,85417327,64140001,77821013,66002498,97869451,74070394,84363635,89770538,79272711,...,87724680,77159757,61272568,72238777,95484688,98977532,137331647,127637758,132485835,148559598
